In [4]:
import torch
import cv2
import numpy as np
import torchvision.transforms as transforms

# Check if GPU is available and use it, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the DehazeNet model architecture (same as during training)
class DehazeNet(torch.nn.Module):
    def __init__(self):  # Corrected the method name
        super(DehazeNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 3, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

# Load the trained model (update the path to where you saved the model on your laptop)
model = DehazeNet().to(device)

try:
    model.load_state_dict(torch.load("D:\\Mini project Git\\Dehazing-and-Human-Detection-Project\\Dehazing_model\\dehaze_model.pth"))  # Path to the saved model file
    print("Model loaded successfully.")
except RuntimeError as e:
    print(f"Error loading the model: {e}")

model.eval()  # Set the model to evaluation mode

# Define the image transformation (same as used during training)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Reverse the normalization after prediction
def reverse_normalization(tensor):
    return tensor * 0.5 + 0.5  # Reverse normalization applied during training

# Function to process frames through the model
def process_frame(frame):
    # Ensure the input frame is in the correct format
    if frame is None or not isinstance(frame, np.ndarray):
        raise ValueError("Invalid frame input. Ensure it's a valid NumPy array.")

    # Preprocess the frame
    input_frame = transform(frame).unsqueeze(0).to(device)  # Add batch dimension
    with torch.no_grad():
        output = model(input_frame).cpu()

    # Postprocess the output
    output_frame = output.squeeze(0).permute(1, 2, 0).numpy()  # CHW to HWC
    output_frame = reverse_normalization(output_frame)  # Reverse normalization
    output_frame = np.clip(output_frame * 255, 0, 255).astype(np.uint8)  # Ensure valid pixel range
    return output_frame


Model loaded successfully.


In [5]:
# Open the webcam feed (ensure your webcam is available and connected)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Dehaze the frame using the trained model
    dehazed_frame = process_frame(frame_rgb)

    # Convert the dehazed frame back to BGR for OpenCV
    dehazed_frame_bgr = cv2.cvtColor(dehazed_frame, cv2.COLOR_RGB2BGR)

    # Display the dehazed frame
    cv2.imshow("Dehazed Frame", dehazed_frame_bgr)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()